# Setup

In [1]:
# !pip install 'crewai[tools]' -q

In [2]:
# %cd /content/drive/MyDrive/AI Agents/CrewAI/Job Interview Prep Part 2

In [ ]:
# Retrieve API Keys and store them as Environment variables
# from google.colab import userdata
# openai_api_key = userdata.get('ai_agents_openai')
# serper_api_key = userdata.get('serper_api')

# import os
# os.environ['OPENAI_API_KEY'] = openai_api_key
# os.environ['SERPER_API_KEY'] = serper_api_key



In [6]:
# import libraries
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [7]:
# Define the LLM
llm = ChatOpenAI(model_name = "gpt-4o", temperature  = 0.5)

# User Inputs

In [8]:
# Call for inputs: Interviewer, company, job position, job description:
interviewer = input("Enter the name of the interviewer (if unknown include None): ")
company = input("Enter the name of the company: ")
job_position = input("Enter the job position: ")
job_description = input("Enter the job description: ")

Enter the name of the interviewer (if unknown include None): Matthias Glöel
Enter the name of the company: Beat81
Enter the job position: Data Analytics Working Student
Enter the job description: About the Role As a Working Student in our Analytics Team, you will work closely with our Tech, Product, Marketing and Operations team to support our daily operations with analytical insights and help us to grow. You will be joining a dynamic, fast-paced start-up where you’ll have ownership and impact from day one!  Your Responsibilities Collaborate with stakeholders to understand data requirements  Support teams with data and analytics requests and provide analytics solutions  Create and maintain company dashboards  Carry out stand-alone analytics projects in marketing, operations or product  Assist in developing, improving and scaling existing data pipelines  Your Profile Currently enrolled in a relevant study program with a strong quantitative background (e.g. mathematics, computer science,

# Research Crew

## Research AI Agent

In [35]:
# Create the Research AI Agent
research_agent = Agent(
    role = "Research Agent",
    goal = "conduct in-depth research",
    backstory = """ As a Research Specialist, your mission is to uncover detailed and relevant information.
    The ultimate goal is to meet the expectations o the task""",
    llm = llm,
    tools = [ScrapeWebsiteTool(),  SerperDevTool()])

## Research Tasks

In [36]:
# Define the task about researching the company
research_company_task = Task(
    description = f"""
    Research about {company}, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months""",
    expected_output = f""" A comprehensive report in Markdown with bullet point about {company} and its indutry""",
    agent = research_agent)

In [37]:
# Research Interviewer Task
research_person_task = Task(
    description = f"Research about {interviewer} with a focus on their professional career",
    expected_output = f"""A comprehensive report in Markdown with bullet point about {interviewer}""",
    agent = research_agent)

## AI Coach + Interview Prep

In [38]:
# Build the AI interview coach agent
coach_agent = Agent(
    role = "Interview Coach",
    goal = f"I help the user prepare for the job interview for job {job_position}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [39]:
# Prepare the lsit of questions
define_questions_task = Task(
    description = f"""
    Define a list a of questions to prepare the user for the job {job_position} based on the company {company}, job description {job_description} and interviewer {interviewer}""",
    expected_output = f""" a list of 20 questions to ask the user in Markdown format that cover: Culture and Team fit, Job position Fit, Background and ways of working, Growth Mindset""",
    agent = coach_agent,
    context = [research_company_task, research_person_task])

## Assemble the Crew

In [40]:
# Assembling the crew
crew = Crew(
    agents = [research_agent, coach_agent],
    tasks = [research_company_task, research_person_task, define_questions_task],
    verbose = True,
    process = Process.sequential)
result = crew.kickoff({"topic": "Write a list of question to prepare for the the interview"})

# Agent: Research Agent
## Task: 
    Research about Beat81, their position in the sector and recent industry trends.
    Focus on recent news and developments from last 12 months


# Agent: Research Agent
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Beat81 recent news 2023\"}"
## Tool Output: 
{'searchParameters': {'q': 'Beat81 recent news 2023', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'BEAT81 | FIBO 2023 - it was a blast   Thanks to everyone who joined ...', 'link': 'https://www.instagram.com/beat81/reel/CrGp6lsORBh/', 'snippet': "We're opening a new studio in Berlin! Celebrate with us on September 12th as we launch our newest HIIT studio in the heart of Friedrichshain.", 'position': 1}, {'title': 'BEAT81 | The countdown is on! Starting today, you have until ...', 'link': 'https://www.instagram.com/beat81/p/CvXPaN-ssI7/', 'snippet': 'Join us on September 12th to celebrate the opening of our newest HIIT fitness st

## Exporting

In [41]:
# Check the crew outcome
display(Markdown(result.tasks_output[2].raw))

```markdown
# Interview Preparation Questions for Data Analytics Working Student at Beat81

## Culture and Team Fit
1. How do you align with Beat81's core values of inclusivity, growth commitment, radical care, and energizing community?
2. Can you describe a time when you contributed to a team culture similar to Beat81's?
3. How do you typically engage with a community, and why is it important in a workplace setting?
4. What motivates you to work in a dynamic, fast-paced start-up environment like Beat81?
5. How do you handle feedback and integrate it into your work process to foster personal and professional growth?

## Job Position Fit
6. What excites you most about the opportunity to work as a Data Analytics Working Student at Beat81?
7. How do you prioritize and manage multiple analytics requests from different teams?
8. Describe your experience with creating and maintaining dashboards. What tools have you used?
9. How do you approach stand-alone analytics projects in marketing, operations, or product?
10. Can you provide an example of how you've improved or scaled a data pipeline in a previous role or project?

## Background and Ways of Working
11. How has your academic background prepared you for this role in data analytics?
12. Discuss your experience with SQL and Python. How have you applied them in past projects?
13. What is your proficiency level with Google Sheets, and how have you utilized it in data analysis?
14. How familiar are you with data visualization tools like Tableau or Lookerstudio?
15. What experience do you have with dbt, GCP, or BigQuery, and how do you plan to leverage them at Beat81?

## Growth Mindset
16. How do you stay updated with the latest trends and developments in data analytics and the fitness industry?
17. Describe a situation where you had to learn a new skill or tool quickly to complete a project.
18. How do you measure success in your analytics work, and what steps do you take to achieve it?
19. Can you share an example of a challenging project and how you overcame obstacles to deliver results?
20. How do you plan to grow your skills and contribute to Beat81's mission of bringing fitness and happiness to everyone's life?

These questions are designed to help you prepare for your interview with Beat81, focusing on understanding your fit with the company culture, the specific job role, your background and working style, and your growth mindset. Good luck with your interview preparation!
```

In [42]:
# Export each of the tasks output
# Exporting the  company report
file_name = f"{company}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[0].raw)

# Export the interviewer report
file_name = f"{interviewer}_report.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[1].raw)

# Export the interview questions
file_name = f"{job_position}_questions.txt"

# Open the file in write mode and write the content
with open(file_name, "w") as file:
    file.write(result.tasks_output[2].raw)

# Interview Prep Crew

In [43]:
# Import the list of questions
with open(f"{job_position}_questions.txt", "r") as file:
    questions = file.read()

In [61]:
# AI Agent to ask questions
interviewer_agent = Agent(
    role = "Interviewer Agent",
    goal = f"I help the user prepare for the job interview for job {job_position} asking a question from {questions}",
    backstory = f"You are an expert in job interviews in companies like {company}",
    llm = llm)

In [62]:
# Define the interview prep task
interview_prep_task = Task(
    description = f"""
    Interview the user for the job {job_position} and ask one question from {questions}
    """,
    expected_output = f"""
    ask only one random question to the user  from {questions}
    """,
    agent = interviewer_agent,
    human_input = True
)

In [63]:
# Define the Coaching Task
coaching_task = Task(
    description = """ Give feedback to the user on its answer """,
    expected_output = f""" Give feedback to the user on its answer with a bullet points on what was good, bad, and how to take it to the next level""",
    agent = coach_agent,
    context = [interview_prep_task],
    human_input = True
)

## Assemble crew

In [54]:
# # Assemble the coaching crew
# crew = Crew(
#     agents = [interviewer_agent, coach_agent],
#     tasks = [interview_prep_task, coaching_task],
#     verbose = True,
#     process = Process.hierarchical,
#     manager_agent = None,
#     manager_llm=ChatOpenAI(temperature=0, model="gpt-4o"),
#     memory = True,
#     planning = True,
#     cache = True)

# result = crew.kickoff({"topic": f"Ask me a question  from {questions} for the job prep"})


 
[2025-02-04 16:01:12][INFO]: Planning the crew execution
# Agent: Crew Manager
## Task: 
    Interview the user for the job Data Analytics Working Student and ask one question from ```markdown
# Interview Preparation Questions for Data Analytics Working Student at Beat81

## Culture and Team Fit
1. How do you align with Beat81's core values of inclusivity, growth commitment, radical care, and energizing community?
2. Can you describe a time when you contributed to a team culture similar to Beat81's?
3. How do you typically engage with a community, and why is it important in a workplace setting?
4. What motivates you to work in a dynamic, fast-paced start-up environment like Beat81?
5. How do you handle feedback and integrate it into your work process to foster personal and professional growth?

## Job Position Fit
6. What excites you most about the opportunity to work as a Data Analytics Working Student at Beat81?
7. How do you prioritize and manage multiple analytics requests from d

KeyboardInterrupt: 

In [64]:
# Assemble the coaching crew
crew = Crew(
    agents = [interviewer_agent, coach_agent],
    tasks = [interview_prep_task, coaching_task],
    verbose = True,
    process = Process.sequential)

result = crew.kickoff({"topic": f"Ask me a question  from {questions} for the job prep"})


# Agent: Interviewer Agent
## Task: 
    Interview the user for the job Data Analytics Working Student and ask one question from ```markdown
# Interview Preparation Questions for Data Analytics Working Student at Beat81

## Culture and Team Fit
1. How do you align with Beat81's core values of inclusivity, growth commitment, radical care, and energizing community?
2. Can you describe a time when you contributed to a team culture similar to Beat81's?
3. How do you typically engage with a community, and why is it important in a workplace setting?
4. What motivates you to work in a dynamic, fast-paced start-up environment like Beat81?
5. How do you handle feedback and integrate it into your work process to foster personal and professional growth?

## Job Position Fit
6. What excites you most about the opportunity to work as a Data Analytics Working Student at Beat81?
7. How do you prioritize and manage multiple analytics requests from different teams?
8. Describe your experience with creat

#LangGraph